In [4]:
pip install transformers

     |████████████████████████████████| 2.0MB 12.5MB/s 
     |████████████████████████████████| 890kB 26.9MB/s 
     |████████████████████████████████| 3.2MB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=26654e28922842a8abca6acb2cbea78d31910ae361a4f77295059a595a377cbe
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


**Função  de Cálculo de Score**

In [9]:
def calc_score_tarefa1(text, target_word, tokenizer, model, debug=False):

  tokenized_text = tokenizer.tokenize(text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  #get indexs
  target_index = tokenizer.convert_tokens_to_ids([target_word])[0]
  masked_index = tokenized_text.index('[MASK]') 

  # Create the segments tensors.
  segments_ids = [0] * len(tokenized_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  
  # Predict all tokens
  with torch.no_grad():
      predictions = model(tokens_tensor, segments_tensors)

  # get words again (just to compare the confidence)
  expected_token = tokenizer.convert_ids_to_tokens([target_index])[0]

  # normalise between 0 and 1
  predictions_candidates = torch.sigmoid(predictions[0][0][masked_index])
  
  predicted_index = torch.argmax(predictions_candidates).item()

  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

  # if word dont exist return 0 
  if predicted_token == '[UNK]':
    return 0

  predictions_candidates = predictions_candidates.cpu().numpy()
  target_bert_confiance = predictions_candidates[target_index]
  predicted_bert_confience = predictions_candidates[predicted_index]


  # 
  score = 1 - (target_bert_confiance-predicted_bert_confience if  target_bert_confiance > predicted_bert_confience else predicted_bert_confience-target_bert_confiance)
  if debug:
    print("predicted token ---> ", predicted_token, predicted_bert_confience)
    print("expected token  ---> ",expected_token , target_bert_confiance)
    print("Score:", score)

  return score

In [10]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)


# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')
model.eval()


text = '[CLS] Especificamente, a capacidade impressionante desses insetos de se espremerem por qualquer espaço e aguentarem [MASK] de até 900 vezes seu próprio peso sem sofrer grandes danos [SEP]'
#text = '[CLS] Chegamos na metade da temporada! Você disputou [MASK] contra todas as equipes do grupo uma vez e a agora inicia-se o returno [SEP]'

target_word = 'pressões'

# calculate score
score = calc_score_tarefa1(text, target_word, tokenizer, model, debug=True)
# print("Score: ", score)a1(text, target_word, tokenizer, model, debug=True)
print("Score:",score)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


predicted token --->  cargas 0.9999845
expected token  --->  pressões 0.999969
Score: 0.9999845027923584
Score: 0.9999845027923584


**BERT - Buscando o Contexto : Tarefa 1**
- Metodo calc_score_tarefa1